# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,houghton,47.1219,-88.5690,3.95,81,84,4.60,US,1712329671
1,1,adamstown,-25.0660,-130.1015,22.04,62,92,3.55,PN,1712329496
2,2,severo-yeniseyskiy,60.3725,93.0408,-3.30,98,100,1.24,RU,1712329567
3,3,grytviken,-54.2811,-36.5092,2.88,89,100,1.82,GS,1712329702
4,4,noosa heads,-26.3833,153.1500,21.73,80,34,1.32,AU,1712329433


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# I like it a bit warmer than the original specifications
# A max temperature lower than 33 degrees but higher than 22
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 33) & (city_data_df["Max Temp"] > 22)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,jose pedro varela,-33.4500,-54.5333,22.19,48,0,2.43,UY,1712329704
58,58,teknaf,20.8624,92.3058,26.51,88,0,2.39,BD,1712329709
148,148,tura,25.5198,90.2201,26.50,27,0,1.46,IN,1712329721
162,162,rio grande,-32.0350,-52.0986,25.06,52,0,3.86,BR,1712329679
192,192,tamanrasset,22.7850,5.5228,30.95,6,0,4.12,DZ,1712329727
240,240,port sudan,19.6175,37.2164,28.01,65,0,4.35,SD,1712329733
248,248,gualeguay,-33.1416,-59.3097,22.50,57,0,2.95,AR,1712329734
286,286,san juan,-31.5375,-68.5364,26.29,41,0,2.03,AR,1712329597
297,297,amalapuram,16.5833,82.0167,27.62,76,0,3.14,IN,1712329739
316,316,pimentel,-6.8367,-79.9342,27.16,61,0,2.06,PE,1712329741


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
hotel_df = city_data_df[["City", "Country", "Max Temp", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Humidity,Hotel Name
16,jose pedro varela,UY,22.19,-33.4500,-54.5333,48,
58,teknaf,BD,26.51,20.8624,92.3058,88,
148,tura,IN,26.50,25.5198,90.2201,27,
162,rio grande,BR,25.06,-32.0350,-52.0986,52,
192,tamanrasset,DZ,30.95,22.7850,5.5228,6,
240,port sudan,SD,28.01,19.6175,37.2164,65,
248,gualeguay,AR,22.50,-33.1416,-59.3097,57,
286,san juan,AR,26.29,-31.5375,-68.5364,41,
297,amalapuram,IN,27.62,16.5833,82.0167,76,
316,pimentel,PE,27.16,-6.8367,-79.9342,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jose pedro varela - nearest hotel: No hotel found
teknaf - nearest hotel: No hotel found
tura - nearest hotel: Hotel Polo Orchid
rio grande - nearest hotel: Hotel Vila Moura Executivo
tamanrasset - nearest hotel: فندق الأمان
port sudan - nearest hotel: Ranya Hotel
gualeguay - nearest hotel: No hotel found
san juan - nearest hotel: Hotel San Francisco
amalapuram - nearest hotel: Homewood Suites
pimentel - nearest hotel: Hospedaje Calle del Sol
navasota - nearest hotel: Best Western
san rafael - nearest hotel: Hotel Regional
bayji - nearest hotel: No hotel found
castillos - nearest hotel: A mi gente
as suwayq - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
shush - nearest hotel: هتل آپادانا
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
murudeshwara - nearest hotel: R N Shetty Hotels
jianshui - nearest hotel: 阳阳大酒店


,City,Country,Max Temp,Lat,Lng,Humidity,Hotel Name
16,jose pedro varela,UY,22.19,-33.4500,-54.5333,48,No hotel found
58,teknaf,BD,26.51,20.8624,92.3058,88,No hotel found
148,tura,IN,26.50,25.5198,90.2201,27,Hotel Polo Orchid
162,rio grande,BR,25.06,-32.0350,-52.0986,52,Hotel Vila Moura Executivo
192,tamanrasset,DZ,30.95,22.7850,5.5228,6,فندق الأمان
240,port sudan,SD,28.01,19.6175,37.2164,65,Ranya Hotel
248,gualeguay,AR,22.50,-33.1416,-59.3097,57,No hotel found
286,san juan,AR,26.29,-31.5375,-68.5364,41,Hotel San Francisco
297,amalapuram,IN,27.62,16.5833,82.0167,76,Homewood Suites
316,pimentel,PE,27.16,-6.8367,-79.9342,61,Hospedaje Calle del Sol


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color="City",
    alpha=.75,
    size = "Humidity",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)